In [91]:
import pandas as pd
import altair as alt
import warnings
warnings.filterwarnings("ignore")
alt.renderers.set_embed_options(theme='light')
pd.set_option("display.max_colwidth", 400)
pd.set_option("display.max_rows", 400)


Trend line of what yLUNA is being used for?  PRISM Farm, yLUNA Staking, LPing, or Nothing.

In [92]:
prism_addr = 'terra1dh9478k2qvqhqeajhn75a2a7dsnf74y5ukregw'
yluna_addr = 'terra17wkadg0tah554r35x6wvff0y5s7ve8npcjfuhz'
pluna_addr = 'terra1tlgelulz9pdkhls6uglfn5lmxarx7f2gxtdzh2'
pLuna_PRISM_Pair = 'terra1persuahr6f8fm6nyup0xjc7aveaur89nwgs5vs'
yLuna_PRISM_Pair = 'terra1kqc65n5060rtvcgcktsxycdt2a4r67q2zlvhce'

In [93]:
def claim(claim_hash):
    df = pd.read_json(
            f"https://api.flipsidecrypto.com/api/v2/queries/{claim_hash}/data/latest",
            convert_dates=["BLOCK_TIMESTAMP"])
    df.columns = [c.lower() for c in df.columns]
    return df

In [94]:
def get_url(url):
    return pd.read_csv(url, index_col=0)

In [95]:
class ChartProvider:
    def __init__(self):
        pass
    
    def get_yluna_time_area_chart(self, df, scale_, min_date=None, max_date=None, top_padding=0):
        max_date = df['Time'].max()
        df['Amount (millions)'] = round(df['Amount']/1000000,2).apply(str)+'M'
        chart = alt.Chart(df).mark_area().encode(
            x=alt.X('Time:T',scale=alt.Scale(domain=(min_date,max_date))),
            y=alt.X('Amount:Q',scale=alt.Scale(domain=(0,df['Amount'].max()+top_padding))),
            color=alt.Color('Type:N', 
                        scale=scale_,
                        legend=alt.Legend(
                                    orient='none',
                                    padding=5,
                                    legendY=0,
                                    direction='vertical')),
            tooltip=[alt.Tooltip('Time:T', format='%Y-%m-%d'),'Type:N','Amount (millions):N']
        )
        return chart

In [96]:
class RefractDataProvider:
    def __init__(self, claim, get_url, path_to_data='../data'):
        self.refract = 'c5ac5e60-7da0-429f-98e8-19ccdd77d835'
        self.refract_cluna = '5b0257c3-e93b-49d4-93f6-e370bc3b3f50'
        self.claim = claim
        self.get_url = get_url
        self.path_to_data = path_to_data
    
    def load(self):
        self.load_from_url()
        self.load_from_csv()
        self.refract_df.columns = [c.lower() for c in self.refract_df.columns]
        self.refract_df_from_csv.columns = [c.lower() for c in self.refract_df_from_csv.columns]
        self.refract_cluna_df.columns = [c.lower() for c in self.refract_cluna_df.columns]
        self.refract_cluna_df_from_csv.columns = [c.lower() for c in self.refract_cluna_df_from_csv.columns]
        self.refract_df = self.refract_df.append(self.refract_df_from_csv).drop_duplicates()
        self.refract_cluna_df = self.refract_cluna_df.append(self.refract_cluna_df_from_csv).drop_duplicates()
            
    def load_from_url(self):
        self.refract_df = self.claim(self.refract)
        self.refract_cluna_df = self.claim(self.refract_cluna)
        
    def write_to_csv(self):
        self.refract_df.to_csv(f'{self.path_to_data}/refract.csv')
        self.refract_cluna_df.to_csv(f'{self.path_to_data}/refract_cluna.csv')
        
    def load_from_csv(self):
        self.refract_df_from_csv = pd.read_csv(f'{self.path_to_data}/refract.csv', index_col=0)
        self.refract_cluna_df_from_csv = pd.read_csv(f'{self.path_to_data}/refract_cluna.csv', index_col=0)
        
    def parse_refracting_cluna(self):
        self.refract_cluna_df.columns = [c.lower() for c in self.refract_cluna_df.columns]
        self.refract_cluna_df['user'] = self.refract_cluna_df['from_']
        self.refract_cluna_df_pol = self.refract_cluna_df[['block_timestamp','tx_id','user','0_action','0_amount']]
        self.refract_cluna_df_pol.columns = ['block_timestamp','tx_id','user','action','amount']
        self.refract_cluna_df_pol['asset_given'] = 'cLUNA'
        self.refract_cluna_df_pol['asset_received'] = 'yLUNA'
        self.refract_cluna_df_pol['operation'] = 'refraction'
        self.refract_cluna_df_pol.block_timestamp=self.refract_cluna_df_pol.block_timestamp.apply(str).apply(lambda x: x[:-4] if len(x) == 23 else x)
        self.refract_cluna_df_pol.block_timestamp=self.refract_cluna_df_pol.block_timestamp.apply(str).apply(lambda x: x[:-3] if len(x) == 22 else x)
        self.refract_cluna_df_pol.block_timestamp=self.refract_cluna_df_pol.block_timestamp.apply(str).apply(lambda x: x[:-7] if len(x) == 26 else x) 
        self.refract_cluna_df_pol['hr'] = self.refract_cluna_df_pol.block_timestamp.str[:-5] + '00:00.000'
        self.refract_cluna_df_pol['day'] = self.refract_cluna_df_pol.block_timestamp.str[:-9]
        self.refract_cluna_df_pol['amount_signed'] = self.refract_cluna_df_pol.amount
        
        
    def parse_refracting_luna(self):
        self.refract_df.columns = [c.lower() for c in self.refract_df.columns]
        self.refract_df['user'] = self.refract_df.apply(lambda row: row.from_ if row['0_action']=='bond_split' else row.to_, axis=1)
        self.refract_df_pol = self.refract_df[['block_timestamp','tx_id','user','0_action','0_amount']]
        self.refract_df_pol.columns = ['block_timestamp','tx_id','user','action','amount']
        self.refract_df_pol['asset_given'] = self.refract_df_pol.apply(lambda row: 'LUNA' if row['action']=='bond_split' else 'yLUNA',axis=1)
        self.refract_df_pol['asset_received'] = self.refract_df_pol.apply(lambda row: 'yLUNA' if row['action']=='bond_split' else 'LUNA',axis=1)
        self.refract_df_pol['operation'] = 'refraction'
        self.refract_df_pol.block_timestamp=self.refract_df_pol.block_timestamp.apply(str).apply(lambda x: x[:-4] if len(x) == 23 else x)
        self.refract_df_pol.block_timestamp=self.refract_df_pol.block_timestamp.apply(str).apply(lambda x: x[:-3] if len(x) == 22 else x)
        self.refract_df_pol.block_timestamp=self.refract_df_pol.block_timestamp.apply(str).apply(lambda x: x[:-7] if len(x) == 26 else x)
        self.refract_df_pol['hr'] = self.refract_df_pol.block_timestamp.str[:-5] + '00:00.000'
        self.refract_df_pol['day'] = self.refract_df_pol.block_timestamp.str[:-9]
        self.refract_df_pol['amount_signed'] = self.refract_df_pol.apply(lambda row: -row.amount 
                                                        if row.action=='burn_from' else row.amount,axis=1)
        
    def parse(self):
        self.parse_refracting_cluna()
        self.parse_refracting_luna()
        self.all_refreact = self.refract_df_pol.append(self.refract_cluna_df_pol)
        daily_delta_rf = self.all_refreact.groupby('day').amount_signed.sum().reset_index()
        daily_delta_rf = daily_delta_rf.sort_values(by='day')
        daily_delta_rf['cumsum'] = daily_delta_rf.amount_signed.cumsum().apply(lambda x: round(x,2))
        daily_delta_rf.columns = ['Time', 'Amount signed', 'Amount']
        daily_delta_rf['Type'] = 'yLuna circulating'
        self.daily_delta_rf = daily_delta_rf


In [97]:
class YLunaStakingDataProvider:
    def __init__(self, claim, get_url, path_to_data='../data'):
        self.ystaking = '3ff0fc49-5a0d-4cdf-a8ab-33f8ea7e755f'
        self.ystaking_farm = '05d91866-0193-4231-b2ca-1774fbd5742a'
        self.claim = claim
        self.get_url = get_url
        self.path_to_data = path_to_data
    
    def load(self):
        self.load_from_url()
        self.load_from_csv()
        self.ystaking_df.columns = [c.lower() for c in self.ystaking_df.columns]
        self.ystaking_df_from_csv.columns = [c.lower() for c in self.ystaking_df_from_csv.columns]
        self.ystaking_farm_df.columns = [c.lower() for c in self.ystaking_farm_df.columns]
        self.ystaking_farm_df_from_csv.columns = [c.lower() for c in self.ystaking_farm_df_from_csv.columns]
        self.ystaking_df = self.ystaking_df.append(self.ystaking_df_from_csv).drop_duplicates()
        self.ystaking_farm_df = self.ystaking_farm_df.append(self.ystaking_farm_df_from_csv)
            
    def load_from_url(self):
        self.ystaking_df = self.claim(self.ystaking)
        self.ystaking_farm_df = self.claim(self.ystaking_farm)
        
    def write_to_csv(self):
        self.ystaking_df.to_csv(f'{self.path_to_data}/ystaking.csv')
        self.ystaking_farm_df.to_csv(f'{self.path_to_data}/ystaking_farm.csv')
        
    def load_from_csv(self):
        self.ystaking_df_from_csv = pd.read_csv(f'{self.path_to_data}/ystaking.csv', index_col=0)
        self.ystaking_farm_df_from_csv = pd.read_csv(f'{self.path_to_data}/ystaking_farm.csv', index_col=0)
        
    def parse_ystaking(self):
        self.ystaking_df.columns = [c.lower() for c in self.ystaking_df.columns]
        self.ystaking_df['action'] = self.ystaking_df.apply(lambda row: row['0_action'] if row['0_action']=='unbond' else row['1_action'],axis=1)
        self.ystaking_df['amount'] = self.ystaking_df.apply(lambda row: row['0_amount'] if row['0_action']=='unbond' else row['1_amount'],axis=1)
        self.ystaking_df['user'] = self.ystaking_df.apply(lambda row: row['to_'] if row.action=='unbond' else row['from_'],axis=1)
        self.ystaking_df['asset_given'] = self.ystaking_df.apply(lambda row: None if row.action=='unbond' else 'yLUNA',axis=1)
        self.ystaking_df['asset_received'] = self.ystaking_df.apply(lambda row: 'yLUNA' if row.action=='unbond' else None,axis=1)
        self.ystaking_df.block_timestamp=self.ystaking_df.block_timestamp.apply(str).apply(lambda x: x[:-4] if len(x) == 23 else x)
        self.ystaking_df.block_timestamp=self.ystaking_df.block_timestamp.apply(str).apply(lambda x: x[:-3] if len(x) == 22 else x)
        self.ystaking_df.block_timestamp=self.ystaking_df.block_timestamp.apply(str).apply(lambda x: x[:-7] if len(x) == 26 else x)
        self.ystaking_df['day'] = self.ystaking_df.hr.apply(str).str[:-13]
        self.ystaking_df = self.ystaking_df[['block_timestamp','tx_id','hr','day','action','amount','user','asset_given','asset_received']]
        self.ystaking_df['amount_signed'] = self.ystaking_df.apply(lambda row: row.amount if row.action=='bond' else -row.amount,axis=1)
        self.ystaking_df = self.ystaking_df.drop_duplicates(ignore_index=True)
        
    def parse_ystaking_farm(self):
        self.ystaking_farm_df.columns = [c.lower() for c in self.ystaking_farm_df.columns]
        self.ystaking_farm_df['action'] = self.ystaking_farm_df.apply(lambda row: row['3_action'] if row['3_action']=='bond' else row['1_action'], axis=1)
        self.ystaking_farm_df['amount'] = self.ystaking_farm_df.apply(lambda row: float(row['0_amount']) if row['action']=='bond' else float(row['0_amount']), axis=1)
        self.ystaking_farm_df = self.ystaking_farm_df[['block_timestamp','tx_id','sender','action','amount']]
        self.ystaking_farm_df.block_timestamp=self.ystaking_farm_df.block_timestamp.apply(str).apply(lambda x: x[:-4] if len(x) == 23 else x)
        self.ystaking_farm_df.block_timestamp=self.ystaking_farm_df.block_timestamp.apply(str).apply(lambda x: x[:-3] if len(x) == 22 else x)
        self.ystaking_farm_df.block_timestamp=self.ystaking_farm_df.block_timestamp.apply(str).apply(lambda x: x[:-7] if len(x) == 26 else x)
        self.ystaking_farm_df['hr'] = self.ystaking_farm_df.block_timestamp.str[:-5] + '00:00.000'
        self.ystaking_farm_df['day'] = self.ystaking_farm_df.block_timestamp.str[:-9]
        self.ystaking_farm_df['amount_signed'] = self.ystaking_farm_df.apply(lambda row: row.amount if row.action=='bond' else -row.amount,axis=1)
        self.ystaking_farm_df = self.ystaking_farm_df.drop_duplicates(ignore_index=True)
    
    def parse(self):
        self.parse_ystaking()
        self.parse_ystaking_farm()

In [98]:
class SwapsDataProvider:
    def __init__(self, claim, get_url, path_to_data='../data'):
        self.swaps = '1bfd8019-89a1-470d-8868-60d71e57d1d0'
        self.router = '69b149fb-81ba-4860-aac1-b17f0d6d7688'
        self.claim = claim
        self.get_url = get_url
        self.path_to_data = path_to_data
            
    def load(self):
        self.load_from_url()
        self.load_from_csv()
        self.swaps_df.columns = [c.lower() for c in self.swaps_df.columns]
        self.swaps_df_from_csv.columns = [c.lower() for c in self.swaps_df_from_csv.columns]
        self.router_df_from_csv.columns = [c.lower() for c in self.router_df_from_csv.columns]
        self.router_df.columns = [c.lower() for c in self.router_df.columns]
        self.swaps_df = self.swaps_df.append(self.swaps_df_from_csv).drop_duplicates()
        self.router_df = self.router_df.append(self.router_df_from_csv).drop_duplicates()
            
    def load_from_url(self):
        self.swaps_df = self.claim(self.swaps)
        self.router_df = self.claim(self.router)
        
    def write_to_csv(self):
        self.swaps_df.to_csv(f'{self.path_to_data}/swaps.csv')
        self.router_df.to_csv(f'{self.path_to_data}/router.csv')
        
    def load_from_csv(self):
        self.swaps_df_from_csv = pd.read_csv(f'{self.path_to_data}/swaps.csv', index_col=0)
        self.router_df_from_csv = pd.read_csv(f'{self.path_to_data}/router.csv', index_col=0)
        
    def parse_simple_swaps(self):
        self.swaps_df.columns = [c.lower() for c in self.swaps_df.columns]
        swaps_df = self.swaps_df[self.swaps_df.ask_asset != '']
        swaps_df = swaps_df[swaps_df.sender.notna()]
        swaps_df_pol = swaps_df.rename(columns={'sender':'user','ask_asset':'asset_received','offer_asset':'asset_given'})
        swaps_df_pol = swaps_df_pol[['block_timestamp','tx_id','price','user','asset_received','return_amount','asset_given','offer_amount']]
        swaps_df_pol['operation'] = 'swap'
        self.swaps_df_pol = swaps_df_pol.drop_duplicates(ignore_index=True)
        
    def parse_router(self):
        self.router_df.columns = [c.lower() for c in self.router_df.columns]
        router_df = self.router_df.rename(columns={'sender':'user'})
        router_df_1 = router_df[['block_timestamp','tx_id','0_ask_asset','0_offer_amount','0_offer_asset','0_price','0_return_amount','user']]
        router_df_1 = router_df_1.rename(columns={'0_ask_asset':'asset_received','0_offer_amount':'offer_amount',
                           '0_price':'price','0_return_amount':'return_amount','0_offer_asset':'asset_given'})
        router_df_2 = router_df[['block_timestamp','tx_id','1_ask_asset','1_offer_amount','1_offer_asset','1_price','1_return_amount','user']]
        router_df_2 = router_df_2.rename(columns={'1_ask_asset':'asset_received','1_offer_amount':'offer_amount',
                           '1_price':'price','1_return_amount':'return_amount','1_offer_asset':'asset_given'})
        router_df_pol = router_df_1.append(router_df_2)
        router_df_pol['operation'] = 'swap'
        self.router_df_pol = router_df_pol.drop_duplicates(ignore_index=True)
    
    def parse(self):
        self.parse_simple_swaps()
        self.parse_router()
        self.swaps_df_all = self.router_df_pol.append(self.swaps_df_pol[self.router_df_pol.columns])
        self.swaps_df_all.block_timestamp=self.swaps_df_all.block_timestamp.apply(str).apply(lambda x: x[:-4] if len(x) == 23 else x)
        self.swaps_df_all.block_timestamp=self.swaps_df_all.block_timestamp.apply(str).apply(lambda x: x[:-3] if len(x) == 22 else x)
        self.swaps_df_all.block_timestamp=self.swaps_df_all.block_timestamp.apply(str).apply(lambda x: x[:-7] if len(x) == 26 else x)
        self.swaps_df_all['hr'] = self.swaps_df_all.block_timestamp.str[:-5] + '00:00.000'
        self.swaps_df_all['day'] = self.swaps_df_all.block_timestamp.str[:-9]
        #
        yluna_swaps = self.swaps_df_all[(self.swaps_df_all.asset_given=='yLUNA')|(self.swaps_df_all.asset_received=='yLUNA')]
        yluna_swaps['amount_signed'] = yluna_swaps.apply(lambda row: row.offer_amount if row.asset_given == 'yLUNA' else -row.return_amount,axis=1)
        yluna_swaps['type'] = 'swap'
        self.yluna_swaps = yluna_swaps.drop_duplicates(ignore_index=True)
        #
        pluna_swaps = self.swaps_df_all[(self.swaps_df_all.asset_given=='pLUNA')|(self.swaps_df_all.asset_received=='pLUNA')]
        pluna_swaps['amount_signed'] = pluna_swaps.apply(lambda row: row.offer_amount if row.asset_given == 'pLUNA' else -row.return_amount,axis=1)
        pluna_swaps['type'] = 'swap'
        self.pluna_swaps = pluna_swaps.drop_duplicates(ignore_index=True)
        

In [99]:
class LPDataProvider:
    def __init__(self, claim, get_url, path_to_data='../data'):
        self.lp_provide_withdraw = '7f5c7008-648a-4944-9fe9-e6c37e2e7bb8'
        self.claim = claim
        self.get_url = get_url
        self.path_to_data = path_to_data
            
    def load(self):
        self.load_from_url()
        self.load_from_csv()
        self.lp_provide_withdraw_df.columns = [c.lower() for c in self.lp_provide_withdraw_df.columns]
        self.lp_provide_withdraw_df_from_csv.columns = [c.lower() for c in self.lp_provide_withdraw_df_from_csv.columns]
        self.lp_provide_withdraw_df = self.lp_provide_withdraw_df.append(self.lp_provide_withdraw_df_from_csv).drop_duplicates()
            
    def load_from_url(self):
        self.lp_provide_withdraw_df = self.claim(self.lp_provide_withdraw)
        
    def write_to_csv(self):
        self.lp_provide_withdraw_df.to_csv(f'{self.path_to_data}/lp_provide_withdraw.csv')
        
    def load_from_csv(self):
        self.lp_provide_withdraw_df_from_csv = pd.read_csv(f'{self.path_to_data}/lp_provide_withdraw.csv', index_col=0)
        
    def get_action(self, row):
        for i in range(-1,6):
            prefix = f"{i}_" if i >= 0 else ""
            if(f'{prefix}action' in row):
                if(row[f'{prefix}action'] in ['provide_liquidity','withdraw_liquidity']):
                    return row[f'{prefix}action']
                
    def get_n_action(self, row):
        for i in range(-1,6):
            prefix = f"{i}_" if i >= 0 else ""
            if(f'{prefix}action' in row):
                if(row[f'{prefix}action'] in ['provide_liquidity','withdraw_liquidity']):
                    return f'{prefix}'
                
    def correct_parsing(self, row):
        if (row.prefix == '1_' and row.f_action == 'withdraw_liquidity') or \
            (row.prefix == '0_' and row.f_action == 'provide_liquidity'):
            return False
        else:
            return True
                
    def parse(self):
        self.lp_provide_withdraw_df.columns = [c.lower() for c in self.lp_provide_withdraw_df.columns]
        df = self.lp_provide_withdraw_df
        df['f_action'] = df.apply(self.get_action,axis=1)
        df['prefix'] = df.apply(self.get_n_action,axis=1)
        df.block_timestamp=df.block_timestamp.apply(str).apply(lambda x: x[:-4] if len(x) == 23 else x)
        df.block_timestamp=df.block_timestamp.apply(str).apply(lambda x: x[:-3] if len(x) == 22 else x)
        df.block_timestamp=df.block_timestamp.apply(str).apply(lambda x: x[:-7] if len(x) == 26 else x) 
        df['f_contract_address'] = df.apply(lambda row: row[f'{row.prefix}contract_address'], axis=1)
        df['asset'] = df.apply(lambda row: 'yLuna' if row.f_contract_address=='terra1kqc65n5060rtvcgcktsxycdt2a4r67q2zlvhce' \
                                    else 'pLuna', axis=1)
        assert df.apply(self.correct_parsing , axis=1).sum() == 0
        provide_ = df[df.f_action=='provide_liquidity']
        withdraw_ = df[df.f_action=='withdraw_liquidity']
        #
        provide_['prism_amount'] = provide_.apply(lambda row: row['2_amount'] if row['1_contract_address'] == prism_addr else row['3_amount'],axis=1)
        provide_['asset_amount'] = provide_.apply(lambda row: row['1_amount'] if row['2_contract_address'] in [yluna_addr,pluna_addr] else row['2_amount'],axis=1)
        provide_['sender'] = provide_['from_']
        provide_['hr'] = provide_.block_timestamp.str[:-5] + '00:00.000'
        provide_['day'] = provide_.block_timestamp.str[:-9]
        provide_ = provide_[['block_timestamp','sender','tx_id','f_action','prism_amount','asset','asset_amount','hr','day']]
        provide_['amount_signed'] = provide_.asset_amount
        provide_['type'] = 'provide_lp'
        self.provide_ = provide_.drop_duplicates(ignore_index=True).drop_duplicates(['tx_id'],ignore_index=True)
        #
        withdraw_['prism_amount'] = withdraw_.apply(lambda row: row['1_amount'] if row['2_contract_address'] == prism_addr else row['2_amount'],axis=1)
        withdraw_['asset_amount'] = withdraw_.apply(lambda row: row['2_amount'] if row['3_contract_address'] in [yluna_addr,pluna_addr] else row['1_amount'],axis=1)
        withdraw_['sender'] = withdraw_['from_']
        withdraw_['hr'] = withdraw_.block_timestamp.str[:-5] + '00:00.000'
        withdraw_['day'] = withdraw_.block_timestamp.str[:-9]
        withdraw_ = withdraw_[['block_timestamp','sender','tx_id','f_action','prism_amount','asset','asset_amount','hr','day']]
        withdraw_['amount_signed'] = -withdraw_.asset_amount
        withdraw_['type'] = 'withdraw_lp'
        self.withdraw_ = withdraw_.drop_duplicates(ignore_index=True).drop_duplicates(['tx_id'],ignore_index=True)

In [100]:
class CollectorDataProvider:
    def __init__(self, claim, get_url, path_to_data='../data'):
        self.collector = '2ab62a07-3882-48e6-bdc6-d9e592aee2d8'
        self.claim = claim
        self.get_url = get_url
        self.path_to_data = path_to_data
            
    def load(self):
        self.load_from_url()
        self.load_from_csv()
        self.collector_df.columns = [c.lower() for c in self.collector_df.columns]
        self.collector_df_from_csv.columns = [c.lower() for c in self.collector_df_from_csv.columns]
        self.collector_df = self.collector_df.append(self.collector_df_from_csv).drop_duplicates()
               
    def load_from_url(self):
        self.collector_df = self.claim(self.collector)
        
    def write_to_csv(self):
        self.collector_df.to_csv(f'{self.path_to_data}/collector.csv')
        
    def load_from_csv(self):
        self.collector_df_from_csv = pd.read_csv(f'{self.path_to_data}/collector.csv', index_col=0)
        
    def get_amount_asset(self,row, pair_addr, asset_addr):
        #Swapping Prism for yLuna
        for i in range(-1,6):
            prefix = f"{i}_" if i >= 0 else ""
            addr = str(row[f'{prefix}offer_asset']).replace('cw20:','') if row[f'{prefix}offer_asset'] else ''
            if(addr in [asset_addr]):
                return row[f'{prefix}offer_amount']
        #Asking for yLuna from the pool
        for i in range(-1,6):
            prefix = f"{i}_" if i >= 0 else ""
            addr = str(row[f'{prefix}ask_asset']).replace('cw20:','') if row[f'{prefix}ask_asset'] else ''
            if(addr in [asset_addr]):
                return -row[f'{prefix}return_amount']
        #Swapping PRISM for yLuna
        for i in range(-1,6):
            prefix = f"{i}_" if i >= 0 else ""
            if(row[f'{prefix}contract_address'] in [pair_addr]):
                if(row[f'{prefix}from'] == [pair_addr]):
                    if(row[f'{prefix}action'] == 'swap'):
                        return row[f'{i}_amount']
        #Sending yLuna to the pool
        for i in range(-1,6):
            prefix = f"{i}_" if i >= 0 else ""
            if(row[f'{prefix}contract_address'] == prism_addr):
                if(row[f'{prefix}from'] in [pair_addr]):
                    if(row[f'{prefix}action'] == 'send'):
                        return row[f'{prefix}amount']
        return None

    def get_contract(self,row, pair_addr, asset_addr):
        #Swapping Prism for yLuna
        for i in range(-1,6):
            prefix = f"{i}_" if i >= 0 else ""
            addr = str(row[f'{prefix}offer_asset']).replace('cw20:','') if row[f'{prefix}offer_asset'] else ''
            if(addr in [asset_addr]):
                return addr
        #Asking for yLuna from the pool
        for i in range(-1,6):
            prefix = f"{i}_" if i >= 0 else ""
            addr = str(row[f'{prefix}ask_asset']).replace('cw20:','') if row[f'{prefix}ask_asset'] else ''
            if(addr in [asset_addr]):
                return addr
        #Swapping PRISM for yLuna
        for i in range(-1,6):
            prefix = f"{i}_" if i >= 0 else ""
            if(row[f'{prefix}contract_address'] in [pair_addr]):
                if(row[f'{prefix}from'] == [pair_addr]):
                    if(row[f'{prefix}action'] == 'swap'):
                        return row[f'{prefix}contract_address']
        #Sending yLuna to the pool
        for i in range(-1,6):
            prefix = f"{i}_" if i >= 0 else ""
            if(row[f'{prefix}contract_address'] == prism_addr):
                if(row[f'{prefix}from'] in [pair_addr]):
                    if(row[f'{prefix}action'] == 'send'):
                        return row[f'{prefix}from']
        return None
    
    def parse_asset(self, df, asset_addr, pair_addr, asset_name):
        df = df.copy()
        df['amount_signed'] = df.apply(self.get_amount_asset,args=(pair_addr, asset_addr),axis=1)/1000000
        df['f_contract'] = df.apply(self.get_contract,args=(pair_addr, asset_addr),axis=1)
        df = df[df.f_contract.notna()]
        df['asset'] = asset_name
        return df
        
        
    def parse(self,withdraw_,provide_,swaps):
        self.lp_txs = withdraw_[['tx_id']].append(provide_[['tx_id']])\
                            .append(swaps[['tx_id']])
        self.lp_txs.columns = [c.lower() for c in self.lp_txs.columns]
        self.collector_df.columns = [c.lower() for c in self.collector_df.columns]
        collector_df = self.collector_df
        collector_df.block_timestamp=collector_df.block_timestamp.apply(str).apply(lambda x: x[:-4] if len(x) == 23 else x)
        collector_df.block_timestamp=collector_df.block_timestamp.apply(str).apply(lambda x: x[:-3] if len(x) == 22 else x)
        collector_df.block_timestamp=collector_df.block_timestamp.apply(str).apply(lambda x: x[:-7] if len(x) == 26 else x) 
        collector_df['hr'] = collector_df.block_timestamp.str[:-5] + '00:00.000'
        collector_df['day'] = collector_df.block_timestamp.str[:-9]
        collector_df = collector_df[collector_df.tx_id.isin(set(self.collector_df.tx_id.unique()).difference(self.lp_txs.tx_id.unique()))]
        collector_df['type'] = 'collector_and_other'
        self.collector_df = collector_df
        df_pluna = self.parse_asset(self.collector_df, pluna_addr, pLuna_PRISM_Pair, 'pLuna')
        df_yluna = self.parse_asset(self.collector_df, yluna_addr, yLuna_PRISM_Pair, 'yLuna')
        self.collector_pyluna = df_pluna.append(df_yluna).drop_duplicates(ignore_index=True)
        
        assert len(self.collector_pyluna) > 0
        assert len(self.collector_pyluna[~(self.collector_pyluna.f_contract.isin(\
                    [yLuna_PRISM_Pair,pLuna_PRISM_Pair,yluna_addr,pluna_addr]))]) == 0
        assert self.collector_pyluna[(self.collector_pyluna.day < '2022-03-10') &
                                     ~(self.collector_pyluna.tx_id.isin(['F1371E3F465062633DF33F66CAA3CD8BF430FB010C38EFDE7F99ECBB4D3F6FD3']))]\
                ['amount_signed'].isna().sum() == 0
        

In [101]:
class DataProvider:
    def __init__(self, asset_name, path_to_data='../data'):
        self.path_to_data = path_to_data
        self.asset_name = asset_name
        self.dates_to_mark = pd.DataFrame([
            ['2022-02-13', '2022-02-13',2000000,'Prism Forge'], 
            ['2022-03-06', '2022-03-06',2400000,'Prism Farm']], 
            columns=['text_date','date','height','text']
        )
    def lp_delta(self, withdraw_, provide_, yluna_swaps, collector_df):
        self.all_lps = withdraw_[['day','amount_signed','tx_id','type','block_timestamp']]\
                            .append(provide_[['day','amount_signed','tx_id','type','block_timestamp']])\
                            .append(yluna_swaps[['day','amount_signed','tx_id','type','block_timestamp']])\
                            .append(collector_df[['day','amount_signed','tx_id','type','block_timestamp']]).drop_duplicates(ignore_index=True)
        daily_delta_lp = self.all_lps.groupby('day').amount_signed.sum().reset_index()
        daily_delta_lp = daily_delta_lp.sort_values(by='day')
        daily_delta_lp['cumsum'] = daily_delta_lp.amount_signed.cumsum().apply(lambda x: round(x,2))
        daily_delta_lp.columns = ['Time', 'Amount signed', 'Amount']
        daily_delta_lp['Type'] = f'{self.asset_name} LP'
        self.daily_delta_lp = daily_delta_lp
        
    def stk_farm_delta(self, ystaking_farm_df):
        daily_delta_stk_farm = ystaking_farm_df.groupby('day').amount_signed.sum().reset_index()
        daily_delta_stk_farm = daily_delta_stk_farm.sort_values(by='day')
        daily_delta_stk_farm['cumsum'] = daily_delta_stk_farm.amount_signed.cumsum().apply(lambda x: round(x,2))
        daily_delta_stk_farm.columns = ['Time', 'Amount signed', 'Amount']
        daily_delta_stk_farm['Type'] = f'{self.asset_name} Farm staked'
        self.daily_delta_stk_farm = daily_delta_stk_farm
        
    def stk_delta(self, ystaking_df):
        daily_delta_stk = ystaking_df.groupby('day').amount_signed.sum().reset_index()
        daily_delta_stk = daily_delta_stk.sort_values(by='day')
        daily_delta_stk['cumsum'] = daily_delta_stk.amount_signed.cumsum().apply(lambda x: round(x,2))
        daily_delta_stk.columns = ['Time', 'Amount signed', 'Amount']
        daily_delta_stk['Type'] = f'{self.asset_name} staked'
        self.daily_delta_stk = daily_delta_stk
        
    def refact_delta(self, all_refreact):
        daily_delta_rf = all_refreact.groupby('day').amount_signed.sum().reset_index()
        daily_delta_rf = daily_delta_rf.sort_values(by='day')
        daily_delta_rf['cumsum'] = daily_delta_rf.amount_signed.cumsum().apply(lambda x: round(x,2))
        daily_delta_rf.columns = ['Time', 'Amount signed', 'Amount']
        daily_delta_rf['Type'] = f'{self.asset_name} circulating'
        self.daily_delta_rf = daily_delta_rf
    
    def unused_asset(self, all_deltas):
        self.asset_used = all_deltas[self.all_deltas.Type.isin([f'{self.asset_name} LP',f'{self.asset_name} staked',f'{self.asset_name} Farm staked'])]
        asset_tot = self.all_deltas[self.all_deltas.Type.isin([f'{self.asset_name} circulating'])]
        asset_unused = self.asset_used.groupby('Time').Amount.sum().rename('Amount_used').reset_index().merge(asset_tot, on='Time')
        asset_unused['Amount_delta'] = asset_unused['Amount'] - asset_unused['Amount_used']
        asset_unused['Amount_delta'] = asset_unused['Amount_delta'].apply(lambda x: x if x>0 else 0)
        asset_unused = asset_unused[['Time','Amount signed','Amount_delta','Type']]
        asset_unused['Type'] = f'{self.asset_name} unused'
        asset_unused.columns = ['Time','Amount signed','Amount','Type']
        self.asset_unused = asset_unused
        
    def all_delta(self):
        self.all_deltas = self.daily_delta_rf.append(self.daily_delta_lp)
        try:
            self.all_deltas = self.all_deltas.append(self.daily_delta_stk)
        except:
            pass
        try:
            self.all_deltas = self.all_deltas.append(self.daily_delta_stk_farm)
        except:
            pass
        
    def fill_date_gaps(self, dff, extra_dates=[]):
        dd = dff.Time.unique()
        dd = [*dd,*extra_dates]
        unique_dates = pd.Series(dd).rename('Time').reset_index().drop(columns='index')
        for t in dff.Type.unique():
            df = dff[dff.Type==t]
            last_date = df.Time.max()
            last_value = df[df.Time==last_date].Amount.values[0]
            df = df.merge(unique_dates, on='Time', how='right')
            df = df[df.Type.isna()]
            df['Type'] = t
            df['Amount'] = df.apply(lambda row: last_value if row.Time>last_date else 0,axis=1)
            dff = dff.append(df.fillna(0))
        return dff


In [102]:
ystake_dp = YLunaStakingDataProvider(claim,get_url,'../data')
#ystake_dp.load_from_url()
ystake_dp.load()
#ystake_dp.write_to_csv()
ystake_dp.parse()

In [103]:
refract_dp = RefractDataProvider(claim,get_url,'../data')
#refract_dp.load_from_url()
refract_dp.load()
#refract_dp.write_to_csv()
refract_dp.parse()

In [104]:
swaps_dp = SwapsDataProvider(claim,get_url,'../data')
#swaps_dp.load_from_url()
swaps_dp.load()
#swaps_dp.write_to_csv()
swaps_dp.parse()

In [105]:
lp_dp = LPDataProvider(claim,get_url,'../data')
#lp_dp.load_from_url()
lp_dp.load()
#lp_dp.write_to_csv()
lp_dp.parse()

In [106]:
df = lp_dp.withdraw_
df[df.tx_id=='C877B30BAC2848C9BFA2255549BEF5CC07C22FBD1E3CD10FE1186E765B8DA786']

,block_timestamp,sender,tx_id,f_action,prism_amount,asset,asset_amount,hr,day,amount_signed,type
59,2022-03-16 16:22:12,terra123rh0aatcewwa5sxl44m8ehf2gy2ruuncen385,C877B30BAC2848C9BFA2255549BEF5CC07C22FBD1E3CD10FE1186E765B8DA786,withdraw_liquidity,0.331685,pLuna,0.009074,2022-03-16 16:00:00.000,2022-03-16,-0.009074,withdraw_lp


In [107]:
collector_dp = CollectorDataProvider(claim,get_url,'../data')
#collector_dp.load_from_url()
collector_dp.load()
#collector_dp.write_to_csv()
collector_dp.parse(lp_dp.withdraw_, lp_dp.provide_, swaps_dp.swaps_df_all)

In [108]:
ydp = DataProvider('yLuna')
ydp.lp_delta(lp_dp.withdraw_[lp_dp.withdraw_.asset=='yLuna'],
            lp_dp.provide_[lp_dp.provide_.asset=='yLuna'], 
            swaps_dp.yluna_swaps, collector_dp.collector_pyluna[collector_dp.collector_pyluna.asset=='yLuna'])
ydp.stk_delta(ystake_dp.ystaking_df)
ydp.stk_farm_delta(ystake_dp.ystaking_farm_df)
ydp.refact_delta(refract_dp.all_refreact)
ydp.all_delta()
#ydp.all_deltas = ydp.fill_date_gaps(ydp.all_deltas)
ydp.unused_asset(ydp.all_deltas)

In [109]:
pdp = DataProvider('pLuna')
pdp.lp_delta(lp_dp.withdraw_[lp_dp.withdraw_.asset=='pLuna'],
            lp_dp.provide_[lp_dp.provide_.asset=='pLuna'], 
            swaps_dp.yluna_swaps, collector_dp.collector_pyluna[collector_dp.collector_pyluna.asset=='pLuna'])
pdp.refact_delta(refract_dp.all_refreact)
pdp.all_delta()
#pdp.all_deltas = pdp.fill_date_gaps(pdp.all_deltas)
pdp.unused_asset(pdp.all_deltas)

### Check pLuna txs

In [110]:
#Check the txs missing for the pool (get complete list from ET)
contract = 'terra1persuahr6f8fm6nyup0xjc7aveaur89nwgs5vs'
txs = pd.read_csv(f'../data/txs/{contract}.csv', index_col=0)
txs.timestamp = pd.to_datetime(txs.timestamp)

In [111]:
all_txs = lp_dp.lp_provide_withdraw_df[['tx_id','block_timestamp']]\
     .append(swaps_dp.swaps_df_all[['tx_id','block_timestamp']])\
     .append(refract_dp.all_refreact[['tx_id','block_timestamp']])\
     .append(collector_dp.collector_pyluna[['tx_id','block_timestamp']])

In [112]:
max_date = min(all_txs.block_timestamp.max(), str(txs.timestamp.max()))

In [113]:
txs = txs[txs.timestamp.apply(str) < max_date]
txs.head()

,hash,timestamp
1,3A66564360F49C4556476EAE4E198553F7A328321E4FD662A6AD39B73FACB599,2022-03-12 15:12:37+00:00
2,6E1356299885C2045A61A4CF5D7B2E8D8BDF22EF8597F2B398E54EF90098E53D,2022-03-12 15:11:46+00:00
3,3D64BB02CF05B1448A6F955C92C12F33C5815DD98221293C81739D9FF8E9080D,2022-03-12 15:10:41+00:00
4,29DBBEB3BF0BF8CDC2862085D9AA2BBD45F6A1FD0A147FF4B5A3D6C5E8A3C077,2022-03-12 15:09:10+00:00
5,08BB2D7527EB367C751708BE4891113403013092B7EDC35FA0D0F177693681C5,2022-03-12 15:08:51+00:00


In [114]:
missing_txs = set(txs.hash.values).difference(set(all_txs.tx_id.values))
#missing_txs
len(missing_txs)

82

### Check LP txs

In [115]:
#Check the txs missing for the pool (get complete list from ET)
contract = 'terra1kqc65n5060rtvcgcktsxycdt2a4r67q2zlvhce'
txs = pd.read_csv(f'../data/txs/{contract}.csv', index_col=0)
txs.timestamp = pd.to_datetime(txs.timestamp)

In [116]:
max_date = min(ydp.all_lps.block_timestamp.max(), str(txs.timestamp.max()))

In [117]:
txs = txs[txs.timestamp.apply(str) < max_date]
txs.head()

,hash,timestamp
1,51DA951E49B713818C95A619D9D2D41DEE6FB09951873E2C8A2842EC8B1338B6,2022-03-10 16:43:12+00:00
2,3A0EE768FCEA4F767E661C1CA9AF6D8B6887DB5699AAB451AE6DD0993937A995,2022-03-10 16:39:59+00:00
3,5DD336F9AEEABFE640141FB170BD9C572B74F577B15434AC34832A7029221412,2022-03-10 16:36:48+00:00
4,A21C0ABECC3C07F8B59CE9921F5116B5F90ED0BBEC698AABD10144989C05AA10,2022-03-10 16:32:03+00:00
5,BA069F0AAF90CFBAEFC68B31F83C346DD3A803FC79326C129894EC3AD858B2C4,2022-03-10 16:29:36+00:00


In [118]:
missing_txs = set(txs.hash.values).difference(set(ydp.all_lps.tx_id.values))
#missing_txs
len(missing_txs)

95

In [119]:
ydp.all_lps[ydp.all_lps.amount_signed.isna()]

,day,amount_signed,tx_id,type,block_timestamp


In [120]:
extra_txs = set(ydp.all_lps.tx_id[(ydp.all_lps.block_timestamp<max_date)&(~ydp.all_lps.type.isin(['withdraw_lp','provide_lp']))].values)\
                .difference(set(txs.hash.values))

In [121]:
len(extra_txs)

2

In [122]:
ydp.all_lps[ydp.all_lps.tx_id.isin(extra_txs)].head()

,day,amount_signed,tx_id,type,block_timestamp
11681,2022-03-10,-0.034511,975E19763B9EAAC9BF2D0E3FD142E18714D77D72DB2C78B7176E5E3581ADFAC4,swap,2022-03-10 16:44:17
19480,2022-03-08,-5.486429,71FC22E26361F196E0935FAA348C7352D534D3557DD9FF58002F082BB696585C,swap,2022-03-08 03:27:52


In [123]:
ydp.all_lps[ydp.all_lps.tx_id.isin(extra_txs)].groupby('type').amount_signed.sum()

type
swap   -5.52094
Name: amount_signed, dtype: float64

## Refract

In [124]:
daily_delta_rf = ydp.daily_delta_rf

In [125]:
daily_delta_rf.head()

,Time,Amount signed,Amount,Type
0,2022-02-14,890610.824139,890610.82,yLuna circulating
1,2022-02-15,213620.887294,1104231.71,yLuna circulating
2,2022-02-16,36275.244129,1140506.96,yLuna circulating
3,2022-02-17,390554.870005,1531061.83,yLuna circulating
4,2022-02-18,19638.781506,1550700.61,yLuna circulating


In [126]:
cp = ChartProvider()
domain = ['yLuna circulating']
range_ = ['#f8936d']
cp.get_yluna_time_area_chart(daily_delta_rf, 
               alt.Scale(domain=domain, range=range_),
               min_date = daily_delta_rf.Time.min(),
               max_date = daily_delta_rf.Time.max(),
               top_padding = 10000
        ).properties(width=800).configure_view(strokeOpacity=0)

alt.Chart(...)

## yStaking

In [127]:
def get_max_domain_date(df, time_field, n_hours):
    if((pd.Timestamp(df[time_field].max()) - 
                pd.Timestamp(df[time_field].min())).total_seconds()/3600 < n_hours):
        max_date = (pd.Timestamp(df[time_field].min()) + pd.to_timedelta(n_hours, unit='h')).strftime("%Y-%m-%dT%H:%M:%SZ")
    else:
        max_date = df[time_field].max()
    return max_date

In [128]:
daily_delta_stk = ydp.daily_delta_stk

In [129]:
domain = ['yLuna staked']
range_ = ['#f8936d']
cp.get_yluna_time_area_chart(daily_delta_stk, 
               alt.Scale(domain=domain, range=range_),
               min_date = daily_delta_stk.Time.min(),
               max_date = daily_delta_stk.Time.max(),
               top_padding = 10000
        ).properties(width=800).configure_view(strokeOpacity=0)

alt.Chart(...)

## yLuna LP

In [130]:
daily_delta_lp = ydp.daily_delta_lp
daily_delta_lp

,Time,Amount signed,Amount,Type
0,2022-02-14,26759.157714,26759.16,yLuna LP
1,2022-02-15,15291.030164,42050.19,yLuna LP
2,2022-02-16,-6100.691928,35949.50,yLuna LP
3,2022-02-17,-7166.866869,28782.63,yLuna LP
4,2022-02-18,-8312.525209,20470.10,yLuna LP
5,2022-02-19,410.850297,20880.95,yLuna LP
6,2022-02-20,-2053.707605,18827.25,yLuna LP
7,2022-02-21,-1491.742902,17335.50,yLuna LP
8,2022-02-22,3597.494742,20933.00,yLuna LP
9,2022-02-23,6059.994139,26992.99,yLuna LP


In [131]:
cp.get_yluna_time_area_chart(daily_delta_lp, 
               alt.Scale(scheme='set2'),
               min_date = daily_delta_stk.Time.min(),
               max_date = daily_delta_stk.Time.max(),
               top_padding = 10000
        ).properties(width=800).configure_view(strokeOpacity=0)

alt.Chart(...)

In [132]:
cp.get_yluna_time_area_chart(ydp.daily_delta_stk_farm, 
               alt.Scale(scheme='set2'),
               min_date = ydp.daily_delta_stk_farm.Time.min(),
               max_date = ydp.daily_delta_stk_farm.Time.max(),
               top_padding = 10000
        ).properties(width=800).configure_view(strokeOpacity=0)

alt.Chart(...)

In [133]:
cp.get_yluna_time_area_chart(ydp.asset_unused, 
               alt.Scale(scheme='set2'),
               min_date = ydp.asset_unused.Time.min(),
               max_date = ydp.asset_unused.Time.max(),
               top_padding = 10000
        ).properties(width=800).configure_view(strokeOpacity=0)

alt.Chart(...)

## All deltas

In [134]:
all_deltas = ydp.asset_used.append(ydp.asset_unused)
all_deltas = ydp.fill_date_gaps(all_deltas, ['2022-02-11','2022-02-12','2022-02-13'])
c1 = cp.get_yluna_time_area_chart(all_deltas, 
               alt.Scale(scheme='set2'),
               min_date = all_deltas.Time.min(),
               max_date = all_deltas.Time.max(),
               top_padding = 1500000
        )

c2 = alt.Chart(ydp.dates_to_mark).mark_rule(color='#e45756').encode(
    x=alt.X('date'+':T',axis=alt.Axis(labels=False,title=''))
)

c3 = alt.Chart(ydp.dates_to_mark).mark_text(
    color='#e45756',
    angle=0
).encode(
    x=alt.X('text_date'+':T',axis=alt.Axis(labels=False,title='')),
    y='height',
    text='text'
)

(c1 + c2 + c3).properties(width=800).configure_view(strokeOpacity=0)

alt.LayerChart(...)

In [135]:
all_deltas = pdp.asset_used.append(pdp.asset_unused)
all_deltas = pdp.fill_date_gaps(all_deltas, ['2022-02-11','2022-02-12','2022-02-13'])
c1 = cp.get_yluna_time_area_chart(all_deltas, 
               alt.Scale(scheme='set2'),
               min_date = all_deltas.Time.min(),
               max_date = all_deltas.Time.max(),
               top_padding = 1500000
        )

c2 = alt.Chart(pdp.dates_to_mark).mark_rule(color='#e45756').encode(
    x=alt.X('date'+':T',axis=alt.Axis(labels=False,title=''))
)

c3 = alt.Chart(pdp.dates_to_mark).mark_text(
    color='#e45756',
    angle=0
).encode(
    x=alt.X('text_date'+':T',axis=alt.Axis(labels=False,title='')),
    y='height',
    text='text'
)

(c1 + c2 + c3).properties(width=800).configure_view(strokeOpacity=0)

alt.LayerChart(...)

## Query

In [136]:
import requests
luna_apr = float(
    requests.get('https://api.terra.dev/chart/staking-return/annualized').json(
    )[-1]['value']) * 100
et_query = requests.get(
    'https://api.extraterrestrial.money/v1/api/prices').json()
yluna_price = float(et_query['prices']['yLUNA']['price'])
luna_price = float(et_query['prices']['LUNA']['price'])
yluna_apr = luna_apr * luna_price / yluna_price

In [137]:
yluna_apr

9.44596017289345

In [138]:
luna_apr

7.130842702631461

In [139]:
luna_price

82.848712435406

In [140]:
df = pd.DataFrame(requests.get('https://api.terra.dev/chart/staking-return/annualized').json())
df['date'] = pd.to_datetime(df['datetime'], unit='ms')
df = df[df['date'] > '2021-07-05 15:00:00']

In [141]:
df.value = df.value.apply(float).apply(lambda x: x*100)

In [142]:
mars_price_chart = alt.Chart(df).mark_line(point = True).encode(
    x=alt.X('date:T'),
    y=alt.X('value:Q'),
    tooltip=[alt.Tooltip('date:T', format='%Y-%m-%d %H:%M'),'value:Q']
).properties(width=800).properties(height=300).configure_view(strokeOpacity=0).interactive()
mars_price_chart

alt.Chart(...)